### Install Boto 3

In [1]:
!pip install boto3==1.34.59

In [1]:
import boto3 as b3

In [2]:
my_br = b3.client(
    service_name="bedrock",
    region_name="us-east-1"
)

In [3]:
list_m = my_br.list_foundation_models()

In [13]:
titan_model_id = list_m['modelSummaries'][6]['modelId']


In [20]:
m_id = titan_model_id.split(":")[0]

In [21]:
m_id

'amazon.titan-text-express-v1'

In [59]:
myprompt = "What is the Avg GDP of India, tell me in one line ?"

In [60]:
import json

In [61]:
my_json_prompt = json.dumps(
    {'inputText': myprompt}
)

In [62]:
my_json_prompt

'{"inputText": "What is the Avg GDP of India, tell me in one line ?"}'

In [63]:
my_br_runtime = b3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"

)

In [64]:
gen_data = my_br_runtime.invoke_model(
    body = my_json_prompt,
    modelId = m_id
)

In [65]:
response_data = gen_data['body'].read()

In [66]:
response_data

b'{"inputTextTokenCount":15,"results":[{"tokenCount":13,"outputText":"\\nThe average GDP of India is $3.2 trillion.","completionReason":"FINISH"}]}'

In [67]:
convert_bin_to_txt = response_data.decode()

In [68]:
convert_to_json = json.loads(convert_bin_to_txt)

In [71]:
convert_to_json

{'inputTextTokenCount': 15,
 'results': [{'tokenCount': 13,
   'outputText': '\nThe average GDP of India is $3.2 trillion.',
   'completionReason': 'FINISH'}]}

In [72]:
print(convert_to_json['results'][0]['outputText'])


The average GDP of India is $3.2 trillion.


## Functions for Agent based Connections

In [1]:
import boto3

client = boto3.client('bedrock-agent', region_name="us-east-1")


In [2]:
client.list_agents() # get agents list, I have executed the code again after deleting agent to confirm that id is removed.

{'ResponseMetadata': {'RequestId': '56c14fef-fb8b-418a-bb2b-fff97b55a900',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Mar 2024 18:17:05 GMT',
   'content-type': 'application/json',
   'content-length': '21',
   'connection': 'keep-alive',
   'x-amzn-requestid': '56c14fef-fb8b-418a-bb2b-fff97b55a900',
   'x-amz-apigw-id': 'UruJVG6aoAMENmw=',
   'x-amzn-trace-id': 'Root=1-65f490a1-5ac25cab616e97bc2bb11c43'},
  'RetryAttempts': 0},
 'agentSummaries': []}

In [28]:
client.list_agent_aliases(agentId='REQLUAAZDQ')

{'ResponseMetadata': {'RequestId': '20d8ffb0-c15d-4109-9157-a9a38e126fe9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Mar 2024 10:44:41 GMT',
   'content-type': 'application/json',
   'content-length': '541',
   'connection': 'keep-alive',
   'x-amzn-requestid': '20d8ffb0-c15d-4109-9157-a9a38e126fe9',
   'x-amz-apigw-id': 'Uqr38FogIAMEGZg=',
   'x-amzn-trace-id': 'Root=1-65f42698-315510c006a7e83d75a74bb0'},
  'RetryAttempts': 0},
 'agentAliasSummaries': [{'agentAliasId': '3HTL1IN8OL',
   'agentAliasName': 'A1',
   'description': 'my alias 1',
   'routingConfiguration': [{'agentVersion': '1'}],
   'agentAliasStatus': 'PREPARED',
   'createdAt': datetime.datetime(2024, 3, 15, 10, 23, 5, 355477, tzinfo=tzutc()),
   'updatedAt': datetime.datetime(2024, 3, 15, 10, 23, 5, 355477, tzinfo=tzutc())},
  {'agentAliasId': 'TSTALIASID',
   'agentAliasName': 'AgentTestAlias',
   'description': 'Test Alias for Agent',
   'routingConfiguration': [{'agentVersion': 'DRAFT'}],
   'agentA

In [65]:
client = boto3.client('bedrock-agent-runtime', region_name="us-east-1")

In [83]:
resp = client.invoke_agent(
    sessionId="A109",
    agentAliasId='agent-alias-id', # use function client.list_agent_aliases(agentId='AgentID')
    enableTrace=False,
    endSession=False,
    agentId='REQLUAAZDQ',
    inputText='lat long for Pune City' # the prompt,)

In [84]:
resp #output

{'ResponseMetadata': {'RequestId': '11e4f133-46ff-4059-b9dc-4ac90364c5be',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Mar 2024 11:46:02 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '11e4f133-46ff-4059-b9dc-4ac90364c5be',
   'x-amz-bedrock-agent-session-id': '47214eca-4bfb-4d8b-a8d0-f1c906700eb4',
   'x-amzn-bedrock-agent-content-type': 'application/json'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'sessionId': '47214eca-4bfb-4d8b-a8d0-f1c906700eb4',
 'completion': <botocore.eventstream.EventStream at 0x1c385400820>}

In [85]:
response_data = resp['completion']

In [86]:
for d in response_data:
    print(d)

{'chunk': {'bytes': b'The latitude and longitude of Pune City are 18.5203 and 73.8567 respectively.'}}


In [97]:
text = d['chunk']['bytes'].decode('utf-8')

# Print the extracted text
print(text)

The latitude and longitude of Pune City are 18.5203 and 73.8567 respectively.


### Doc Link: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/list_agents.html